## Bonus Lab: Batch Transform for on-demand inference

This section provides an introduction to the Amazon SageMaker Batch Transform functionality, which are good for the following scenarios:
- On-demand model evaluations: if you want to run one time model evaluations. For example, to compare accuracy of our trained model on new validation data that we collected after our initial training job.
- Transitional model output: we may want to use outputs from one model as the inputs to another. For example, we may want use a pre-processing step like word embeddings, principal components, clustering, or TF-IDF, before training a second model to generate predictions from that information.
- Periodic inference: in some cases, the inference is scheduled to run periodically at a certain time (e.g., nightly batch inference). In this case, running inference via Batch Transform will save cost as compared to hosting an SageMaker inference endpoint, which runs 24/7.

Functionally, batch transform uses the same mechanics as real-time hosting to generate predictions. It requires a web server that takes in HTTP POST requests a single observation, or mini-batch, at a time. However, unlike real-time hosted endpoints which have persistent hardware (instances stay running until you shut them down), batch transform clusters are torn down when the job completes.

## Instructions

This is a continuation of Lab 4 after the model was created. You will need to update the model name in this notebook. If you don't have a model created, please review Lab 4 (Model deployment), Section "Create model"

In [ ]:
# cell 1

import boto3
import sagemaker
import pandas as pd
import re

from time import gmtime, strftime

In [ ]:
# cell 2

# Set Region
region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)
s3_client = boto3.client("s3", region_name=region)

# Get SageMaker client, role and session
sagemaker_boto_client = boto_session.client("sagemaker")
sagemaker_role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_boto_client
)

Set parameterized variables:
* bucket - S3 Bucket name. You can adjust the code to use a bucket of your choice.
* prefix - String which will be used to identify different resources.
* model name - String which will be used to identify the model.

In [ ]:
# cell 3

# Bucket 
bucket = sagemaker_session.default_bucket()
prefix = "fraud-detect-demo"
model_name = ADD_YOUR_MODEL_NAME_HERE
print("Model name : {}".format(model_name))

#### Prepare data input for Batch Transform job
We will prepare batch data input by removing the predicted value column and other columns, the index, and header row from raw test data

In [ ]:
# cell 4

dataset = pd.read_csv("./data/claims_customer.csv")
batchinput = dataset.drop(["fraud", "Unnamed: 0", "policy_id"], axis=1)
batchinput.head()

# Save back to CSV without index and header (pre-requisites for Batch Transform job data input)
batchinput.to_csv("data/batchinput.csv", index=False, header=False)

# Upload to S3 bucket
s3_client.upload_file(Filename="data/batchinput.csv", Bucket=bucket, Key=f"{prefix}/batch_transform/input/batchinput.csv")

#### Create and run Batch Transform job

In [ ]:
%%time

# cell 5

# Batch Transform to evaluate trained model with test data
batch_job_name = f"{prefix}-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())  # use input data without ID column
batch_input = f"s3://{bucket}/{prefix}/batch_transform/input/batchinput.csv"
batch_output = "s3://{}/{}/batch_transform/output/{}".format(bucket, prefix, batch_job_name)

request = {
    "TransformJobName": batch_job_name,
    "ModelName": model_name,
    "MaxConcurrentTransforms": 6,
    "BatchStrategy": "MultiRecord",
    "TransformOutput": {
        "S3OutputPath": batch_output,
        "Accept": "text/csv",
        "AssembleWith": "Line",
    },
    "TransformInput": {
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": batch_input}},
        "ContentType": "text/csv",
        "SplitType": "Line",
        "CompressionType": "None",
    },
    "TransformResources": {"InstanceType": "ml.m4.xlarge", "InstanceCount": 1},
}

response = sagemaker_boto_client.create_transform_job(**request)
print("Created Transform job with name: ", batch_job_name)

# Wait until the job finishes
try:
    sagemaker_boto_client.get_waiter("transform_job_completed_or_stopped").wait(TransformJobName=batch_job_name)
finally:
    response = sagemaker_boto_client.describe_transform_job(TransformJobName=batch_job_name)
    status = response["TransformJobStatus"]
    print("Transform job ended with status: " + status)
    if status == "Failed":
        message = response["FailureReason"]
        print("Transform failed with the following error: {}".format(message))
        raise Exception("Transform job failed")

#### Reading the prediction results from Batch Transform job

The Batch Transform job outputs the batch prediction results in the S3 location defined in 'batch_output'. We will merge the prediction column with the original sample input dataset.

In [ ]:
# cell 6

def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)    
    s3_client.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

y_predict = get_csv_output_from_s3(batch_output, "batchinput.csv")
y_predict.columns = ['prediction']
predict = pd.concat([y_predict.round().astype(int), dataset], axis=1)
predict.drop(["Unnamed: 0"], axis=1, inplace=True)
predict.head()

Congratulations! You've successfully completed the bonus lab - Batch Transform for on-demand inference